

---


Nombre de los integrantes


*   Cruz Pérez Joshua Santiago
*   Hernández Banda Oziel
*   Jimenez Borzani Naomi Daniela
*   Paredes Hernández Ximena


---


# **EJERCICIO 26**

### **Funciones Auxiliares**

In [ ]:
import numpy as np

# ------------------------- FUNCIONES AUXILIARES ------------------------------ #
def SubMat(Mat, ren, col):
    """Obtiene la submatriz eliminando un renglón y columna específicos."""
    return np.delete(np.delete(Mat, ren, axis=0), col, axis=1)

def Det(Mat):
    """Calcula el determinante de una matriz cuadrada de manera recursiva."""
    # Caso base: matriz 1x1
    if Mat.shape[0] == 1:
        return Mat[0,0]

    # Caso recursivo
    deter = 0.0
    for col in range(Mat.shape[0]):
        deter += ((-1) ** col) * Mat[0][col] * Det(SubMat(Mat, 0, col))
    return deter

def Transpuesta(Mat):
    """Calcula la transpuesta de una matriz cuadrada."""
    transpuesta = np.copy(Mat)
    for ren in range(Mat.shape[0]):
        for col in range(Mat.shape[1]):
            if ren < col:
                transpuesta[ren, col], transpuesta[col, ren] = transpuesta[col, ren], transpuesta[ren, col]
    return transpuesta

# ------------------------- FUNCIONES DE VERIFICACIÓN ------------------------- #
def es_simetrica(A, tol=1e-8):
    """Verifica si una matriz es simétrica."""
    A_t = Transpuesta(np.copy(A))
    for i in range(A.shape[0]):
        for j in range(A.shape[1]):
            if abs(A[i,j] - A_t[i,j]) > tol:
                return False
    return True

def es_definida_positiva(A):
    """Verifica si una matriz es definida positiva."""
    n = A.shape[0]
    for k in range(1, n+1):
        submatriz = A[:k, :k]
        if Det(submatriz) <= 0:
            return False
    return True

# ----------------------- FUNCIONES CHOLESKY ---------------------------------- #
def Cholesky(A):
    """
    Realiza la descomposición de Cholesky clásica de una matriz simétrica definida positiva.

    Parámetros
    ----------
    A : numpy.ndarray
        Matriz cuadrada simétrica y definida positiva de dimensión (n, n)

    Retorna
    -------
    L : numpy.ndarray
        Matriz triangular inferior con diagonal positiva
    L.T : numpy.ndarray
        Matriz triangular superior (transpuesta de L)

    Notas
    -----
    - Implementación directa del algoritmo de Cholesky-Banachiewicz
    - Devuelve matrices tales que A = L @ L.T (factorización LLᵀ)
    - Requiere que la matriz sea:
      * Simétrica exactamente (A = A.T)
      * Definida positiva (autovalores > 0)
    - Complejidad computacional: O(n³/3)
    """
    n = A.shape[0]
    L = np.zeros_like(A)

    for i in range(n):
        for j in range(i+1):
            if i == j:
                sum = 0.0
                for k in range(i):
                    sum += L[i][k] * L[i][k]
                L[i][i] = np.sqrt(A[i][i] - sum)
            else:
                sum = 0.0
                for k in range(j):
                    sum += L[i][k] * L[j][k]
                L[i][j] = (A[i][j] - sum) / L[j][j]
    return L, L.T


def Cholesky2(A):
    """
    Versión alternativa de la descomposición de Cholesky usando eliminación Gaussiana.

    Parámetros
    ----------
    A : numpy.ndarray
        Matriz cuadrada simétrica y definida positiva de dimensión (n, n)

    Retorna
    -------
    L : numpy.ndarray
        Matriz triangular inferior con diagonal unitaria
    U : numpy.ndarray
        Matriz triangular superior
    D : numpy.ndarray
        Matriz diagonal (transpuesta de L en este caso)

    Notas
    -----
    - Implementación basada en la factorización LDLᵀ
    - Devuelve matrices tales que A = L @ U @ L.T (donde U es diagonal)
    - Más estable numéricamente para matrices casi singulares
    - Requiere simetría pero no almacenamiento explícito de ambas mitades
    """
    n = A.shape[0]
    L = np.zeros_like(A)
    U = np.copy(A)
    L = np.eye(A.shape[0])
    aux = []

    for i in range(n):
        Li = np.eye(n)
        for j in range(i+1, n):
            Li[j,i] = -U[j,i]/U[i,i]
            L[j,i] = U[j,i]/U[i,i]
        U = Li @ U
        aux.append(np.copy(Li.T))

    for i in range(len(aux)):
        U = U @ aux[i]

    return L, U, L.T

# ------------------------- MATRICES DE LA PRACTICA --------------------------- #
matrices = {
    "Matriz 1": np.array([
        [2, -1, 0],
        [-1, 2, -1],
        [0, -1, 2]
    ]),
    "Matriz 2": np.array([
        [4, 1, 1, 1],
        [1, 3, -1, 1],
        [1, -1, 2, 0],
        [1, 1, 0, 2]
    ]),
    "Matriz 3": np.array([
        [4, 1, -1, 0],
        [1, 3, -1, 0],
        [-1, -1, 5, 2],
        [0, 0, 2, 4]
    ]),
    "Matriz 4": np.array([
        [6, 2, 1, -1],
        [2, 4, 1, 0],
        [1, 1, 4, -1],
        [-1, 0, -1, 3]
    ])
}

## **Inciso a)**

In [ ]:
# ------------------------ IMPLEMENTACIÓN DE CHOLESKY ------------------------- #
def probar_matrices():
    for nombre, matriz in matrices.items():
        print(f"\n❄ {nombre}:")
        print(matriz)

        # Verificar propiedades
        simetrica = es_simetrica(matriz)
        definida_pos = es_definida_positiva(matriz) if simetrica else False

        print(f"\nCONDICIONES DEL TEOREMA:")
        print(f"  - Simétrica: {'Sí' if simetrica else 'No'}")
        print(f"  - Definida positiva: {'Sí' if definida_pos else 'No'}")

        # Aplicar Cholesky si es posible
        if simetrica and definida_pos:
            print("\nDescomposición Cholesky Clásica (LLᵀ):")
            try:
                L, LT = Cholesky(matriz)
                print("L:")
                print(L)
                print("Lᵀ:")
                print(LT)
            except Exception as e:
                print(f"Error en Cholesky clásico: {str(e)}")

        else:
            print("\nNo se puede aplicar Cholesky")

# Ejecutar pruebas
if __name__ == "__main__":
    probar_matrices()


❄ Matriz 1:
[[ 2 -1  0]
 [-1  2 -1]
 [ 0 -1  2]]

CONDICIONES DEL TEOREMA:
  - Simétrica: Sí
  - Definida positiva: Sí

Descomposición Cholesky Clásica (LLᵀ):
L:
[[ 1  0  0]
 [-1  1  0]
 [ 0 -1  1]]
Lᵀ:
[[ 1 -1  0]
 [ 0  1 -1]
 [ 0  0  1]]

❄ Matriz 2:
[[ 4  1  1  1]
 [ 1  3 -1  1]
 [ 1 -1  2  0]
 [ 1  1  0  2]]

CONDICIONES DEL TEOREMA:
  - Simétrica: Sí
  - Definida positiva: Sí

Descomposición Cholesky Clásica (LLᵀ):
L:
[[ 2  0  0  0]
 [ 0  1  0  0]
 [ 0 -1  1  0]
 [ 0  1  1  0]]
Lᵀ:
[[ 2  0  0  0]
 [ 0  1 -1  1]
 [ 0  0  1  1]
 [ 0  0  0  0]]

❄ Matriz 3:
[[ 4  1 -1  0]
 [ 1  3 -1  0]
 [-1 -1  5  2]
 [ 0  0  2  4]]

CONDICIONES DEL TEOREMA:
  - Simétrica: Sí
  - Definida positiva: Sí

Descomposición Cholesky Clásica (LLᵀ):
L:
[[ 2  0  0  0]
 [ 0  1  0  0]
 [ 0 -1  2  0]
 [ 0  0  1  1]]
Lᵀ:
[[ 2  0  0  0]
 [ 0  1 -1  0]
 [ 0  0  2  1]
 [ 0  0  0  1]]

❄ Matriz 4:
[[ 6  2  1 -1]
 [ 2  4  1  0]
 [ 1  1  4 -1]
 [-1  0 -1  3]]

CONDICIONES DEL TEOREMA:
  - Simétrica: Sí
  - Definida po

## **Inciso b)**

In [ ]:

# ------------------------- IMPLEMENTACIÓN DE CHOLESKY 2------------------------ #
def probar_matrices():
    for nombre, matriz in matrices.items():
        print(f"\n❄ {nombre}:")
        print(matriz)

        # Verificar propiedades
        simetrica = es_simetrica(matriz)
        definida_pos = es_definida_positiva(matriz) if simetrica else False

        print(f"\nCONDICIONES DEL TEOREMA:")
        print(f"  - Simétrica: {'Sí' if simetrica else 'No'}")
        print(f"  - Definida positiva: {'Sí' if definida_pos else 'No'}")

        # Aplicar Cholesky si es posible
        if simetrica and definida_pos:
            print("\nDescomposición Cholesky Alternativa (LDLᵀ):")
            try:
                L_ldl, U, D = Cholesky2(matriz)
                print("L:")
                print(L_ldl)
                print("U (D):")
                print(U)
                print("D (Lᵀ):")
                print(D)
            except Exception as e:
                print(f"Error en Cholesky alternativo: {str(e)}")
        else:
            print("\nNo se puede aplicar Cholesky_2")

# Ejecutar pruebas
if __name__ == "__main__":
    probar_matrices()


❄ Matriz 1:
[[ 2 -1  0]
 [-1  2 -1]
 [ 0 -1  2]]

CONDICIONES DEL TEOREMA:
  - Simétrica: Sí
  - Definida positiva: Sí

Descomposición Cholesky Alternativa (LDLᵀ):
L:
[[ 1.          0.          0.        ]
 [-0.5         1.          0.        ]
 [ 0.         -0.66666667  1.        ]]
U (D):
[[2.         0.         0.        ]
 [0.         1.5        0.        ]
 [0.         0.         1.33333333]]
D (Lᵀ):
[[ 1.         -0.5         0.        ]
 [ 0.          1.         -0.66666667]
 [ 0.          0.          1.        ]]

❄ Matriz 2:
[[ 4  1  1  1]
 [ 1  3 -1  1]
 [ 1 -1  2  0]
 [ 1  1  0  2]]

CONDICIONES DEL TEOREMA:
  - Simétrica: Sí
  - Definida positiva: Sí

Descomposición Cholesky Alternativa (LDLᵀ):
L:
[[ 1.          0.          0.          0.        ]
 [ 0.25        1.          0.          0.        ]
 [ 0.25       -0.45454545  1.          0.        ]
 [ 0.25        0.27272727  0.07692308  1.        ]]
U (D):
[[4.         0.         0.         0.        ]
 [0.         2.75    